In [366]:
import sys
import os
import numpy as np
PLNX_LIBS_PATH = "/Users/alex/Dev_projects/MyOwnRepo/poloniex_api/src"
RT_LIBS_PATH = "/Users/alex/Dev_projects/MyOwnRepo/rt_libs/src"
BA_LIBS_PATH = "/Users/alex/Dev_projects/MyOwnRepo/basic_application/src"
sys.path.append(PLNX_LIBS_PATH)
sys.path.append(RT_LIBS_PATH)
sys.path.append(BA_LIBS_PATH)

In [447]:
from basic_application import with_exception
from basic_application import with_debug_time

In [1]:
import requests
import datetime
import time
import pytz
import numpy as np
import pandas as pd

In [367]:
import time

import numpy as np
import pandas as pd

from data_providers import ClickHouseConnector
from data_providers import DbDataProviderUniversal

In [6]:
import logging
logger = logging.getLogger(__name__)

ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)

logger.addHandler(ch)

logger.setLevel(logging.DEBUG)

In [7]:
#logging.basicConfig(level=logging.INFO)

logger.debug("debug")
logger.info("info")
logger.warning("warning")
logger.error("error")
logger.critical("critical")

debug
info
warning
error
critical


In [4]:
del logger.handlers[0]

In [8]:
logger.handlers

[<StreamHandler stderr (DEBUG)>]

# API

In [474]:
class PublicApiError(Exception):
    def __init__(self, command=None, code=None, message="No message"):
        self.code = code
        self.message = message
        super().__init__(self.message)
        

class PublicApiV2:
    HOST = "https://api.poloniex.com"
    INTERVALS = {
        "MINUTE_1": 60, 
        "MINUTE_5": 300, 
        "MINUTE_10": 600, 
        "MINUTE_15": 900, 
        "MINUTE_30": 1800, 
        "HOUR_1": 3600, 
        "HOUR_2": 7200, 
        "HOUR_4": 14400, 
        "HOUR_6": 21600, 
        "HOUR_12": 43200, 
        "DAY_1": 86400, 
        "DAY_3": 3 * 86400, 
        "WEEK_1": 7 * 86400, 
        "MONTH_1": 30 * 86400    
    }    
    
    @with_exception(PublicApiError)
    def get_interval(self, key):
        return self.INTERVALS[key]
        
    
    def _execute(self, command):
        url = self.HOST + command
        logger.debug("Execute command:  {}".format(url))
        try:
            response = requests.get(url).json()
        except Exception as e:
            raise PublicApiError(command, message=e) from e
        else:
            if "code" in response:
                logger.error(response)
                raise PublicApiError(command, **response)
            else:   
                return response
        
    @with_exception(PublicApiError)
    def get_price(self, symbol):
        path = "/markets/{0}/price".format(symbol)
        response = self._execute(path)
        logger.debug("get_price return: {}".format(response))
        return response

    @with_exception(PublicApiError)
    def get_orderbook(self, symbol, scale=-1, limit=10):
        """
        Get the order book for a given symbol. Scale and limit values are optional.
        endpoint: /markets/{symbol}/orderBook
        @params
        - symbol: String [1] - symbol name
        - scale: String	[0..1] - controls aggregation by price
        - limit: Integer	[0..1] - maximum number of records returned.
        The default value of limit is 10. Valid limit values are: 5, 10, 20, 50, 100, 150.

        @return
        """
        path = "/markets/{0}/orderBook?scale={1}&limit={2}".format(symbol, scale, limit)
        response = self._execute(path)
        
        asks_keys = response["asks"][::2]
        asks_vals = map(float, response["asks"][1::2])
        asks = dict(zip(asks_keys, asks_vals))
        logger.debug("Orderbook asks len: {}".format(len(asks)))

        bids_keys = response["bids"][::2]
        bids_vals = map(float, response["bids"][1::2])
        bids = dict(zip(bids_keys, bids_vals))
        logger.debug("Orderbook bids len: {}".format(len(bids)))
        
        return asks, bids
    @with_exception(PublicApiError)
    def get_trades(self, symbol, limit=500):
        """
        endpoint: /markets/{symbol}/trades
        @params
        - symbol: String [1] - symbol name
        - limit: Integer [0..1] - maximum number of records returned. Default value is 500, and max value is 1000.
        @return
        """
        path = "/markets/{0}/trades?limit={1}".format(symbol, limit)
        response = self._execute(path)
        return response
    
    @with_exception(PublicApiError)
    def get_candles(self, symbol, interval, limit=100, startTime=None, endTime=None):
        """
        endpoint: /markets/{symbol}/candles
        @params
        - symbol: String [1] - symbol name
        - interval: String [1] - the unit of time to aggregate data by. 
        Valid values: MINUTE_1, MINUTE_5, MINUTE_10, MINUTE_15, MINUTE_30, HOUR_1, HOUR_2, HOUR_4, HOUR_6, HOUR_12, DAY_1, DAY_3, WEEK_1 and MONTH_1
        - limit: Integer [0..1] - maximum number of records returned. The default value is 100 and the max value is 500.
        - startTime: Long [0..1] - filters by time. The default value is 0.
        - endTime: Long [0..1] - filters by time. The default value is current time

        """ 
        path = "/markets/{0}/candles?interval={1}&limit={2}".format(symbol, interval, limit)
        if startTime is not None:
            path = path + "&startTime={0}".format(int(startTime))
            
        if endTime is not None:
            path = path + "&endTime={0}".format(int(endTime))
        
        data = self._execute(path)
        columns = ["low", "high", "open", "close", "amount", "quantity", "buyTakerAmount",
        "buyTakerQuantity", "tradeCount", "ts", "weightedAverage", "interval", "startTime", "closeTime"]

        df = pd.DataFrame(data, columns=columns, dtype=float)
        df["sellTakerAmount"] = df["amount"] - df["buyTakerAmount"] 
        df["sellTakerQuantity"] = df["quantity"] - df["buyTakerQuantity"] 

        df["ts"] = df["ts"].astype(int)
        df["startTime"] = df["startTime"].astype(int)
        df["closeTime"] = df["closeTime"].astype(int)
        df["tradeCount"] = df["tradeCount"].astype(int)
        
        df["symbol"] = symbol
        
        columns = ['symbol', 'low', 'high', 'open', 'close', 'amount', 'quantity', 'buyTakerAmount',
       'buyTakerQuantity', 'sellTakerAmount', 'sellTakerQuantity', 'tradeCount', 'weightedAverage', 'interval',
       'startTime', 'closeTime']
        logger.debug("Candles shape: {0} x {1}".format(*df.shape))
        logger.debug("Min ts: {0} | Max ts: {1}".format(
            self.unix_ts_to_date(df["startTime"].min()),
            self.unix_ts_to_date(df["startTime"].max())))
                     
        return df[columns]
    
    def date_to_unix_ts_in_utc(self, date):
        if isinstance(date, str):
            timezone = pytz.timezone("UTC")
            without_timezone = datetime.datetime.strptime(date, "%Y-%m-%d %H:%M:%S")
            with_timezone = timezone.localize(without_timezone)
            return int(with_timezone.timestamp() * 1000)
        else:
            return int(date)

    def unix_ts_to_date(self, unix_ts):
        return datetime.datetime.utcfromtimestamp(unix_ts/1000.0).strftime('%Y-%m-%d %H:%M:%S.%f')

### Тест торгов через кэндлстикс

In [475]:
# Test PublicApi

api = PublicApiV2()
start_dt = "2022-08-15 17:59:00"
end_dt = "2022-08-15 18:00:00"


end_dt = int((time.time()-60)*1000)
start_dt = 300000

start_ts = api.date_to_unix_ts_in_utc(start_dt)
end_ts = api.date_to_unix_ts_in_utc(end_dt) 

symbol = "ETH_USDT"
interval = "MINUTE_1"

df = api.get_candles(symbol, interval, limit=3,  startTime=start_ts, endTime=end_ts)

display(df.shape)
display(df.head(20))

display(api.unix_ts_to_date(min(df["startTime"])))
display(api.unix_ts_to_date(max(df["startTime"])))

#print(api.get_price(symbol))
#print(api.get_orderbook(symbol, limit=500))
#print(api.get_trades(symbol, limit=500))


Execute command:  https://apiaa.poloniex.com/markets/ETH_USDT/candles?interval=MINUTE_1&limit=3&startTime=300000&endTime=1661548628699


PublicApiError: HTTPSConnectionPool(host='apiaa.poloniex.com', port=443): Max retries exceeded with url: /markets/ETH_USDT/candles?interval=MINUTE_1&limit=3&startTime=300000&endTime=1661548628699 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7fb61aa84c40>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))

### Тест торгов

In [42]:
api = PublicApiV2()

symbol = "BTC_USDT"
limit = "1000"


data = api.get_trades(symbol, limit=limit)

Execute command:  https://api.poloniex.com/markets/BTC_USDT/trades?limit=1000


In [57]:
df =  pd.DataFrame(data)
df["price"] = df["price"].astype(float)
df["quantity"] = df["quantity"].astype(float)
df["amount"] = df["amount"].astype(float)
df["ts"] = df["ts"].astype(int)
df["createTime"] = df["createTime"].astype(int)

In [58]:
df.head()

,id,price,quantity,amount,takerSide,ts,createTime
0,60227718,21512.11,0.004936,106.183775,BUY,1661453432077,1661453432072
1,60227717,21511.82,0.000068,1.462804,SELL,1661453428721,1661453428716
2,60227716,21509.53,0.026307,565.851206,SELL,1661453391462,1661453391458
3,60227715,21509.54,0.058426,1256.716384,BUY,1661453383164,1661453383160
4,60227714,21504.08,0.026551,570.954828,BUY,1661453375309,1661453375304


In [59]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          1000 non-null   object 
 1   price       1000 non-null   float64
 2   quantity    1000 non-null   float64
 3   amount      1000 non-null   float64
 4   takerSide   1000 non-null   object 
 5   ts          1000 non-null   int64  
 6   createTime  1000 non-null   int64  
dtypes: float64(3), int64(2), object(2)
memory usage: 54.8+ KB


### Тест Orderbook

In [11]:
api = PublicApiV2()
symbol = "ETH_USDT"
scale = -1
limit = 5

asks, bids = api.get_orderbook(symbol, scale=scale, limit=limit)

display(len(asks))
display(asks)

display(len(bids))
display(bids)

Execute command:  https://api.poloniex.com/markets/ETH_USDT/orderBook?scale=-1&limit=5
Orderbook asks len: 5
Orderbook bids len: 5


5

{'1685.47': 36.84098,
 '1685.48': 0.03,
 '1686.11': 0.525101,
 '1686.13': 2.4,
 '1686.21': 0.349}

5

{'1682.5': 2.925533,
 '1681.79': 2.4,
 '1681.78': 0.97,
 '1681.61': 0.474,
 '1675': 20.0}

# Trade provider classes

## AbstractDataProvider

In [448]:
class AbstractSymbolHandler:
    DATA_INSERT_QUERY = "INSERT INTO table_name VALUES"
    GET_MAX_TS_QUERY = "SELECT MAX(ts) FROM table_name WHERE symbol=%(symbol)s"
    
    def __init__(self, symbol, conn):
        self.symbol = symbol
        self.conn = conn
        self.last_update = 0
        self.ts = 0
        
    @with_debug_time
    def update(self):
        self.ts = int(time.time())
        # step 1 - get data from stock
        df = self._get_data()
        # step 2 - format data
        data = self._transform_data(df)
        # step 3 - sava data
        self._store_data(data)
        
    def _get_data(self):
        pass
    
    def _transform_data(self, df):
        pass
        
    def _store_data(self, data):
        if len(data):
            self.conn.cursor.executemany(self.DATA_INSERT_QUERY, data)
            row_count = self.conn.cursor.rowcount
            logger.info("{0}: {1} orderbook records was written into db".format(self.symbol, row_count))
            self.last_update = self.ts
        else:
            logger.info("{0}: No data to write".format(symbol))
            row_count = 0

        return row_count
    
    def _get_last_update_from_db(self):
        params = {"symbol": self.symbol}
        self.conn.cursor.execute(self.GET_MAX_TS_QUERY, parameters=params)
        raw_data = self.conn.cursor.fetchall()
        max_ts = raw_data[0][0]
        logger.debug("{0} max ts in db: {1}".format(symbol, max_ts))
        return max_ts    

## TradeProvider

In [449]:
class TradeProvider(AbstractSymbolHandler):
    DATA_INSERT_QUERY = "INSERT INTO trades VALUES"
    GET_MAX_TS_QUERY = "SELECT MAX(ts) FROM trades WHERE symbol=%(symbol)s"
    
    LIMIT = 100
    LIMIT_MAX = 1000
    COLUMNS = ["symbol", "ts", "takerSide", "price", "quantity", "amount", "id"]
    
    def __init__(self, symbol, conn):
        super().__init__(symbol, conn)
        self.api = PublicApiV2()

    def _get_data(self):
        """
        Получение данных об объемах с биржи
        """
        if not self.last_update:
            self.last_update = self._get_last_update_from_db()
            
        ts = int(time.time())
        if ts - self.last_update > 300:
            limit = self.LIMIT_MAX
        else:
            limit = self.LIMIT
        data = api.get_trades(self.symbol, limit=limit)
                              
        df = pd.DataFrame(data)
        df["price"] = df["price"].astype(float)
        df["quantity"] = df["quantity"].astype(float)
        df["amount"] = df["amount"].astype(float)
        df["ts"] = df["ts"].astype(int)/1000
        df["ts"] = df["ts"].astype(int)    
        df["createTime"] = df["createTime"].astype(int)
        df["symbol"] = symbol
        
        df = df[df["ts"] > self.last_update]
        df = df.reindex(columns=self.COLUMNS)
        if len(df):
            self.last_update = df["ts"].max()
        return df  

    def _transform_data(self, df):
        return list(df.values)

In [454]:
db_connect_params = {
    "host" : "217.197.116.177",
    "port" : 59000,
    "user" : "alex",
    "password" : "Xrxcmr758",
    "database" : "rt5_dev"
}

with ClickHouseConnector(db_connect_params) as conn:
    
    symbol = "BTC_USDT"
    tp = TradeProvider(symbol, conn)
    tp.update()

Cursor created, database connection established
BTC_USDT max ts in db: 1661547224
Execute command:  https://api.poloniex.com/markets/BTC_USDT/trades?limit=100
BTC_USDT: 1 orderbook records was written into db
Cursor closed


update          | Exec time : 0.33463407


In [430]:
tp.last_update

1661543656

## OrderbookProvider

In [455]:
class OrderbookProvider(AbstractSymbolHandler):
    QUERY = "INSERT INTO orderbook VALUES"
    DATA_INSERT_QUERY = "INSERT INTO orderbook VALUES"
    GET_MAX_TS_QUERY = "SELECT MAX(ts) FROM orderbook WHERE symbol=%(symbol)s"
    
    CHECK_MAX_TS = False
    
    DIVIDER = 1000
    LEVELS_DEFAULT = [0, 1, 2, 3, 5, 8, 13]
    TEMPL = "{:.12f}"
    
    def __init__(self, symbol, conn, levels=(0, 1, 2, 3, 5, 8, 13)):
        super().__init__(symbol, conn)
        self.api = PublicApiV2()
        self.levels = levels
    
    def _get_data(self):
        """Получение данных биржи"""
        price_data = self.api.get_price(symbol)
        price = float(price_data["price"])
 
        asks, bids = self.api.get_orderbook(symbol, scale=-1, limit=150)        
        return asks, bids
    
    def _transform_data(self, data):
        """Преобразование данных для запислив БД"""
        asks, bids = data
        if len(asks) and len(bids):
            lowest_ask = min(map(float, asks.keys()))
            highest_bid = max(map(float, bids.keys()))
        
            record = [
                    self.symbol,
                    self.ts,
                    lowest_ask,
                    highest_bid,
                    asks,
                    bids
                ]
            data = [record]
        else:
            data = []
        return data

In [458]:
db_connect_params = {
    "host" : "217.197.116.177",
    "port" : 59000,
    "user" : "alex",
    "password" : "Xrxcmr758",
    "database" : "rt5_dev"
}

with ClickHouseConnector(db_connect_params) as conn:
    
    symbol = "BTC_USDT"
    tp = OrderbookProvider(symbol, conn)
    tp.update()

Cursor created, database connection established
Execute command:  https://api.poloniex.com/markets/BTC_USDT/price
get_price return: {'symbol': 'BTC_USDT', 'price': '20670.88', 'time': 1661547249931, 'dailyChange': '-0.0452', 'ts': 1661547249936}
Execute command:  https://api.poloniex.com/markets/BTC_USDT/orderBook?scale=-1&limit=150
Orderbook asks len: 150
Orderbook bids len: 150
BTC_USDT: 1 orderbook records was written into db
Cursor closed


update          | Exec time : 0.36476398


In [439]:
d = [0, 1]

a, b = d

In [441]:
b

1

## TradeProviderCandleSticks

In [356]:
class TradeProviderCandleSticks(AbstractDataProvider):
    QUERY = "INSERT INTO volumes VALUES"
    DEPTH = 360
    LIMIT = 3
    INTERVAL = "MINUTE_1"
    
    def __init__(self, conn):
        super().__init__(conn)
        self.api = PublicApiV2()
        
        self.last_update = 0
        self.interval = interval * 1000
        
    def update(self, ts, symbol):
        """
        Основной метод для обновления данных по инструменту
        """
        # step 1 - get data from stock
        df = self._get_data(symbol, ts)
        # step 2 - format data
        data = df.values
        # step 3 - sava data
        #self.save_data(symbol, data)
        return df
        
    def _get_period(self, ts, only_completed_intervals=True):  
        """
        Метод рассчитывает период загрузки - начало и конец для заданного интервала.
        Среди прочего может считать данных из БД (реализовать!)
        
        """
        # Параметры текущего шага
        interval_sec = self.api.get_interval(self.INTERVAL) * 1000
        start_ts = int(np.floor(ts/interval_sec)*interval_sec)
        if only_completed_intervals:
            start_ts = start_ts - interval_sec
        end_ts = start_ts + interval_sec - 1
        logger.debug("current_ts: {0}".format(start_ts))
        
        # Прочие параметры
        next_after_last_upd = self.last_update + interval_sec
        next_after_last_db = 0
        next_max_depth = start_ts - self.DEPTH * 1000
        start_ts = max(next_after_last_upd, next_after_last_db, next_max_depth)
        
        logger.debug("start_ts: {0} | end_ts: {1}".format(start_ts, end_ts))
        return start_ts, end_ts
    

    def _get_data(self, symbol, ts, only_completed_intervals=True):
        """
        Получение данных об объемах с биржи
        """
        
        start_ts, end_ts = self._get_period(ts, only_completed_intervals=only_completed_intervals)
        
        if start_ts <= self.last_update:
            logger.info("Data is already up to date")
        
        else:
            done = False
            data = pd.DataFrame()
            while not done:
                logger.debug("New batch request {0} - {1}".format(self.api.unix_ts_to_date(start_ts), self.api.unix_ts_to_date(end_ts)))
                batch = api.get_candles(symbol, interval, limit=self.LIMIT,  startTime=start_ts, endTime=end_ts)
                
                logger.debug("data len {}".format(len(batch)))
                if len(batch):
                    data = pd.concat([data, batch])
                    # сколько то получили
                    batch_ts_min = batch["startTime"].min()
                    logger.debug("batch_ts_min {}" % batch_ts_min)
                    if batch_ts_min == start_ts:
                        done = True
                    else:
                        logger.debug("real period {0} - {1}".format(self.api.unix_ts_to_date(batch_ts_min), self.api.unix_ts_to_date(end_ts)))
                        end_ts = batch_ts_min - 1
                else:
                    # новая пачка без данных
                    done = True
                    
                logger.debug("=====================")
                    
                # check end condition
            
            data.sort_values(by=['startTime'], inplace=True)
            
            data.reset_index(inplace=True, drop=True)

            return data
        


### Test trade provider class

In [357]:
tp = TradeProviderCandleSticks(None)
symbol = "LTC_USDT"
ts = int(time.time()*1000)
df = tp.update(ts, symbol)

current_ts: 1661458440000
start_ts: 1661458080000 | end_ts: 1661458499999
New batch request 2022-08-25 20:08:00.000000 - 2022-08-25 20:14:59.999000
Execute command:  https://api.poloniex.com/markets/LTC_USDT/candles?interval=MINUTE_1&limit=3&startTime=1661458080000&endTime=1661458499999
Candles shape: 3 x 17
Min ts: 2022-08-25 20:12:00.000000 | Max ts: 2022-08-25 20:14:00.000000
data len 3
batch_ts_min {}
real period 2022-08-25 20:12:00.000000 - 2022-08-25 20:14:59.999000
New batch request 2022-08-25 20:08:00.000000 - 2022-08-25 20:11:59.999000
Execute command:  https://api.poloniex.com/markets/LTC_USDT/candles?interval=MINUTE_1&limit=3&startTime=1661458080000&endTime=1661458319999
Candles shape: 3 x 17
Min ts: 2022-08-25 20:09:00.000000 | Max ts: 2022-08-25 20:11:00.000000
data len 3
batch_ts_min {}
real period 2022-08-25 20:09:00.000000 - 2022-08-25 20:11:59.999000
New batch request 2022-08-25 20:08:00.000000 - 2022-08-25 20:08:59.999000
Execute command:  https://api.poloniex.com/mar

In [358]:
df

,symbol,low,high,open,close,amount,quantity,buyTakerAmount,buyTakerQuantity,sellTakerAmount,sellTakerQuantity,tradeCount,weightedAverage,interval,startTime,closeTime
0,LTC_USDT,57.172,57.229,57.200,57.202,276.271850,4.829653,206.232346,3.605628,70.039504,1.224025,8,57.203261,MINUTE_1,1661458080000,1661458139999
1,LTC_USDT,57.114,57.174,57.174,57.114,370.767510,6.490812,144.248349,2.525373,226.519161,3.965439,9,57.121903,MINUTE_1,1661458140000,1661458199999
2,LTC_USDT,57.079,57.132,57.102,57.132,169.705983,2.971977,169.705983,2.971977,0.000000,0.000000,4,57.102061,MINUTE_1,1661458200000,1661458259999
3,LTC_USDT,57.071,57.136,57.071,57.123,439.955500,7.702593,358.058842,6.269048,81.896659,1.433545,12,57.117852,MINUTE_1,1661458260000,1661458319999
4,LTC_USDT,57.115,57.237,57.149,57.201,462.605958,8.092081,266.997806,4.669872,195.608152,3.422209,12,57.167774,MINUTE_1,1661458320000,1661458379999
5,LTC_USDT,57.223,57.248,57.235,57.247,238.254094,4.162214,118.223185,2.065525,120.030908,2.096689,7,57.242154,MINUTE_1,1661458380000,1661458439999
6,LTC_USDT,57.196,57.245,57.234,57.230,286.619600,5.009666,151.432430,2.646604,135.187170,2.363062,10,57.213320,MINUTE_1,1661458440000,1661458499999


In [125]:
df

In [87]:
tp = TradeProvider(None)

In [88]:
symbol = "ETH_USDT"
ts = int(time.time()*1000)
df = tp.update(symbol, ts)

current_ts: 1661200140000
start_ts: 1661199780000 | end_ts: 1661200199999
New batch request 2022-08-22 20:23:00.000000 - 2022-08-22 20:29:59.999000
Execute command:  https://api.poloniex.com/markets/ETH_USDT/candles?interval=MINUTE_1&limit=3&startTime=1661199780000&endTime=1661200199999
Candles shape: 3 x 16
Min ts: 2022-08-22 20:27:00.000000 | Max ts: 2022-08-22 20:29:00.000000
data len 3
batch_ts_min {}
real period 2022-08-22 20:27:00.000000 - 2022-08-22 20:29:59.999000
New batch request 2022-08-22 20:23:00.000000 - 2022-08-22 20:26:59.999000
Execute command:  https://api.poloniex.com/markets/ETH_USDT/candles?interval=MINUTE_1&limit=3&startTime=1661199780000&endTime=1661200019999
Candles shape: 3 x 16
Min ts: 2022-08-22 20:24:00.000000 | Max ts: 2022-08-22 20:26:00.000000
data len 3
batch_ts_min {}
real period 2022-08-22 20:24:00.000000 - 2022-08-22 20:26:59.999000
New batch request 2022-08-22 20:23:00.000000 - 2022-08-22 20:23:59.999000
Execute command:  https://api.poloniex.com/mar

In [89]:
df

,low,high,open,close,amount,quantity,buyTakerAmount,buyTakerQuantity,sellTakerAmount,sellTakerQuantity,tradeCount,ts,weightedAverage,interval,startTime,closeTime
0,1583.94,1584.26,1584.09,1583.94,118.752536,0.074960,74.458722,0.047000,44.293814,0.027960,11,1661200069217,1584.212063,MINUTE_1,1661200020000,1661200079999
1,1584.07,1585.26,1584.26,1584.07,4495.146861,2.837061,4441.354234,2.803128,53.792628,0.033933,9,1661200140217,1584.437869,MINUTE_1,1661200080000,1661200139999
2,1584.38,1585.75,1584.38,1585.11,70.438361,0.044445,25.685964,0.016199,44.752397,0.028246,5,1661200196217,1584.843562,MINUTE_1,1661200140000,1661200199999
0,1583.96,1584.36,1583.96,1584.25,42.401536,0.026765,10.573953,0.006675,31.827582,0.020090,4,1661199878580,1584.215809,MINUTE_1,1661199840000,1661199899999
1,1582.51,1584.10,1583.32,1584.10,202.086097,0.127647,15.197528,0.009596,186.888568,0.118051,9,1661199954580,1583.163912,MINUTE_1,1661199900000,1661199959999
2,1584.90,1585.74,1584.90,1585.10,84.163415,0.053083,2.225200,0.001404,81.938216,0.051679,5,1661200005217,1585.506071,MINUTE_1,1661199960000,1661200019999
0,1583.09,1585.94,1585.94,1583.47,186.488736,0.117668,91.088855,0.057480,95.399881,0.060188,10,1661199840217,1584.872527,MINUTE_1,1661199780000,1661199839999


## OrderbookProvider

In [382]:
class OrderbookProvider(AbstractDataProvider):
    QUERY = "INSERT INTO orderbook VALUES"
    #MAX_TS_QUERY = "SELECT MAX(ts) FROM orderbook WHERE symbol={0}"
    
    DIVIDER = 1000
    LEVELS_DEFAULT = [0, 1, 2, 3, 5, 8, 13]
    TEMPL = "{:.12f}"
    
    def __init__(self, conn, levels=(0, 1, 2, 3, 5, 8, 13)):
        super().__init__(conn)
        self.api = PublicApiV2()
        self.levels = levels
        
    def update(self, ts, symbol):
        """
        Основной метод для обновления данных по инструменту
        """
        # step 1 - get data from stock
        asks, bids = self.get_orderbook(symbol)
        # step 2 - format data
        data = self._make_db_data(ts, symbol, asks, bids)
        # step 3 - sava data
        self.save_data(symbol, data)
        return data
        
    def get_scale(self, price):
        """
        Рассчитываем значение scale для округления значений стакана. 
        Работает не всегда точно. Пока не использую, пишу все полученные значения
        """
        if  price > 1:
            price_len = len(str(int(price)))
            scale = int('1' + '0'* (price_len))/self.DIVIDER
        else:
            price_len = len(str(int(1/price)))
            scale = 1/(int('1' + '0'* (price_len))*self.DIVIDER)
        return scale
    
    def get_orderbook(self, symbol, auto_scale=False):
        """Получение данных биржи"""
        price_data = self.api.get_price(symbol)
        price = float(price_data["price"])
        
        if auto_scale:
            scale = self.get_scale(price)
        else:
            scale=-1
        
        asks, bids = self.api.get_orderbook(symbol, scale=scale, limit=150)        
        return asks, bids
    
    def _make_db_data(self, ts, symbol, asks, bids):
        """Преобразование данных для запислив БД"""
        if len(asks) and len(bids):
            lowest_ask = min(map(float, asks.keys()))
            highest_bid = max(map(float, bids.keys()))
        
            record = [
                    symbol,
                    ts,
                    lowest_ask,
                    highest_bid,
                    asks,
                    bids
                ]
            data = [record]
        else:
            data = []

        return data
    
    
    def get_orderbook_resampled(self, asks, bids):
        """
        Альтернативный путь уменьшения размера стакана - локальное семплирование
        Пока не использую.
        """
        keys_asks = np.array(list(map(float, asks.keys())))
        vols_asks = np.array(list(map(float, asks.values())))
        keys_bids = np.array(list(map(float, bids.keys())))
        vols_bids = np.array(list(map(float, bids.values())))

        lowest_ask = min(keys_asks)
        highest_bid = max(keys_bids)

        asks_resampled = dict()
        key = self.TEMPL.format(lowest_ask)
        asks_resampled[key] = asks[str(lowest_ask)]

        bids_resampled = dict()
        key = self.TEMPL.format(highest_bid)
        bids_resampled[key] = bids[str(highest_bid)]

        for idx in np.arange(1, len(self.levels)):
            mask_ask = (keys_asks > lowest_ask * (1 + self.levels[idx - 1] / 100.)) & (
                    keys_asks <= lowest_ask * (1 + self.levels[idx] / 100.))
            value = np.round(sum(vols_asks[mask_ask]), 8)
            key = self.TEMPL.format(lowest_ask * (1 + self.levels[idx] / 100.))
            asks_resampled[key] = value

            mask_bid = (keys_bids < highest_bid * (1 - self.levels[idx - 1] / 100.)) & (
                    keys_bids >= highest_bid * (1 - self.levels[idx] / 100.))
            value = np.round(sum(vols_bids[mask_bid]), 8)
            key = self.TEMPL.format(highest_bid * (1 - self.levels[idx] / 100.))
            bids_resampled[key] = value

        return asks_resampled, bids_resampled
    


### Test OrderbookProvider

In [385]:
db_connect_params = {
    "host" : "217.197.116.177",
    "port" : 59000,
    "user" : "alex",
    "password" : "Xrxcmr758",
    "database" : "rt5_dev"
}

with ClickHouseConnector(db_connect_params) as conn:
    
    
    obp = OrderbookProvider(conn)
    symbol = "BTC_USDT"
    ts = int(time.time())
    data = obp.update(ts, symbol)

    

Cursor created, database connection established
Execute command:  https://api.poloniex.com/markets/BTC_USDT/price
get_price return: {'symbol': 'BTC_USDT', 'price': '21591.93', 'time': 1661460063461, 'dailyChange': '-0.0051', 'ts': 1661460063466}
Execute command:  https://api.poloniex.com/markets/BTC_USDT/orderBook?scale=-1&limit=150
Orderbook asks len: 150
Orderbook bids len: 150
BTC_USDT: 1 orderbook records was written into db
Cursor closed


In [375]:
data

[[1661459540142,
  'BTC_USDT',
  21566.52,
  21566.51,
  {'21566.52': 0.75,
   '21566.6': 0.000556,
   '21569.87': 0.375,
   '21572.03': 0.362092,
   '21574.97': 0.375,
   '21575.5': 0.221,
   '21576': 0.8,
   '21576.35': 0.14,
   '21577.87': 0.369984,
   '21580.21': 0.375,
   '21580.24': 0.729081,
   '21582.93': 0.373447,
   '21582.96': 0.002,
   '21585.4': 0.08309,
   '21587.64': 0.375,
   '21591.11': 0.002514,
   '21591.6': 0.443,
   '21591.93': 0.375,
   '21592.31': 0.213811,
   '21594.09': 0.374295,
   '21595.9': 0.08313,
   '21596.25': 0.375,
   '21597.68': 0.002243,
   '21598.41': 0.374869,
   '21604.1': 1.46,
   '21605.99': 0.375,
   '21608.25': 0.375,
   '21610.42': 0.002235,
   '21610.43': 0.375,
   '21612.61': 0.375,
   '21614.79': 0.375,
   '21617.02': 0.375,
   '21617.81': 0.426605,
   '21643.11': 0.64416,
   '21664': 0.01,
   '21694.04': 4.401,
   '21700': 0.001,
   '21750': 0.034636,
   '21750.3': 0.00014,
   '21771': 0.01,
   '21776.5': 0.038891,
   '21800': 0.00375,
  

In [362]:

api = PublicApiV2()

ts = int(time.time()*1000)
obp = OrderbookProvider(None)
symbol = "BTC_USDT"
obp.update(ts, symbol)


Execute command:  https://api.poloniex.com/markets/BTC_USDT/price
get_price return: {'symbol': 'BTC_USDT', 'price': '21589.4', 'time': 1661458589618, 'dailyChange': '-0.0075', 'ts': 1661458589623}
Execute command:  https://api.poloniex.com/markets/BTC_USDT/orderBook?scale=-1&limit=150
Orderbook asks len: 150
Orderbook bids len: 150


[[1661458607305, 'BTC_USDT', 21589.41, 21589.4, {'21589.41': 0.75, '21593.2': 0.221, '21594.09': 0.375, '21594.9': 0.14, '21596.25': 0.375, '21598.41': 0.374869, '21600.1': 0.8, '21601.68': 0.002, '21603.9': 0.443, '21604.78': 1.332886, '21605.99': 0.375, '21606.3': 0.07705, '21608.25': 0.375, '21610.43': 0.375, '21611.7': 0.07714, '21612.61': 0.375, '21614.79': 0.375, '21617.02': 0.375, '21620.35': 0.374739, '21624.37': 0.30386, '21628.37': 0.002235, '21628.38': 0.375, '21629.7': 1.46, '21630': 0.000554, '21636.48': 0.373522, '21639.88': 0.375, '21642.23': 0.375, '21643.53': 0.58677, '21644.4': 0.375, '21664': 0.01, '21664.69': 0.532902, '21694.04': 4.401, '21700': 0.001, '21750': 0.034636, '21771': 0.01, '21776.5': 0.038891, '21800': 0.00375, '21824.19': 2.305, '21841.78': 0.005401, '21850': 4.5e-05, '21861.93': 0.000549, '21889': 5e-05, '21894': 0.03, '21900': 0.990466, '21913.54': 6.7e-05, '21922.84': 0.008502, '21930': 0.01, '21957.25': 7.6e-05, '21965': 0.001047, '21979.19': 9.7e

In [363]:
data[0]

[1661458607305,
 'BTC_USDT',
 21589.41,
 21589.4,
 {'21589.41': 0.75,
  '21593.2': 0.221,
  '21594.09': 0.375,
  '21594.9': 0.14,
  '21596.25': 0.375,
  '21598.41': 0.374869,
  '21600.1': 0.8,
  '21601.68': 0.002,
  '21603.9': 0.443,
  '21604.78': 1.332886,
  '21605.99': 0.375,
  '21606.3': 0.07705,
  '21608.25': 0.375,
  '21610.43': 0.375,
  '21611.7': 0.07714,
  '21612.61': 0.375,
  '21614.79': 0.375,
  '21617.02': 0.375,
  '21620.35': 0.374739,
  '21624.37': 0.30386,
  '21628.37': 0.002235,
  '21628.38': 0.375,
  '21629.7': 1.46,
  '21630': 0.000554,
  '21636.48': 0.373522,
  '21639.88': 0.375,
  '21642.23': 0.375,
  '21643.53': 0.58677,
  '21644.4': 0.375,
  '21664': 0.01,
  '21664.69': 0.532902,
  '21694.04': 4.401,
  '21700': 0.001,
  '21750': 0.034636,
  '21771': 0.01,
  '21776.5': 0.038891,
  '21800': 0.00375,
  '21824.19': 2.305,
  '21841.78': 0.005401,
  '21850': 4.5e-05,
  '21861.93': 0.000549,
  '21889': 5e-05,
  '21894': 0.03,
  '21900': 0.990466,
  '21913.54': 6.7e-05,
  

In [18]:
asks_resampled, bids_resampled = obp.get_orderbook_resampled(asks, bids)

In [19]:
asks_resampled

{'21741.169999999998': 0.759649,
 '21958.581699999999': 12.840231,
 '22175.993399999999': 73.363466,
 '22393.405100000000': 3.726632,
 '22828.228499999997': 1.533618,
 '23480.463599999999': 4.308001,
 '24567.522099999995': 0.786868}

In [20]:
bids_resampled

{'21741.160000000000': 0.622249,
 '21523.748400000000': 19.812866,
 '21306.336800000001': 2.213701,
 '21088.925199999998': 2.925069,
 '20654.101999999999': 4.205795,
 '20001.867200000001': 2.775654,
 '18914.809200000000': 0}

In [662]:
1-23441.03000000/22972.19960000

-0.020408598574078063

In [71]:
api = PublicAPI()

In [60]:
def _convert(value):
    try:
        converted = int(value)
    except Exception:
        converted = 0
    return converted


def _make_db_data(ticker, trades):
    data = []

    for trade in trades:
        order_number = _convert(trade.get("orderNumber", 0))
        g_trade_id = _convert(trade.get("globalTradeID", 0))
        trade_id = _convert(trade.get("tradeID", 0))

        if order_number is not None:
            order_number

        record = [
            api.date_to_unix_ts_in_utc(trade.get("date")),
            ticker,
            trade.get("type", "No_info"),
            float(trade.get("rate", 0.)),
            float(trade.get("amount", 0.)),
            float(trade.get("total", 0.)),
            order_number,
            g_trade_id,
            trade_id
        ]
        data.append(record)
    return data

In [64]:
data_length_total = 0

for batch in api.get_trade_history_batch(ticker, start_ts, end_ts):
    data = _make_db_data(ticker, batch)
    if len(data):
        #self.conn.cursor.executemany(self.PUT_DATA_QUERY, data)
        #data_length = self.conn.cursor.rowcount
        data_length = len(data)
        data_length_total += data_length
        print("{0}: {1} records was written into db".format(ticker, data_length))
    else:
        print("{0}: No data to write".format(ticker))
last_updates[ticker] = ts_end

1660089600 -> 1660089900
Download data from 1660089600 to 1660089900
len response: 100
first record: {'globalTradeID': '60256329', 'tradeID': '60256329', 'date': '2022-08-11 08:20:57', 'type': '1', 'rate': '0.99988', 'amount': '157.59248663', 'total': '157.6114', 'orderNumber': None}
last record: {'globalTradeID': '60256424', 'tradeID': '60256424', 'date': '2022-08-11 08:23:18', 'type': '1', 'rate': '0.9996', 'amount': '43.29157644', 'total': '43.3089', 'orderNumber': None}
USDT_USDD: 100 records was written into db
new end: 2022-08-11 08:23:18
new end ts: 1660206197
done: False
Download data from 1660089600 to 1660206197
len response: 100
first record: {'globalTradeID': '60256329', 'tradeID': '60256329', 'date': '2022-08-11 08:20:57', 'type': '1', 'rate': '0.99988', 'amount': '157.59248663', 'total': '157.6114', 'orderNumber': None}
last record: {'globalTradeID': '60256424', 'tradeID': '60256424', 'date': '2022-08-11 08:23:18', 'type': '1', 'rate': '0.9996', 'amount': '43.29157644', '

len response: 100
first record: {'globalTradeID': '60256329', 'tradeID': '60256329', 'date': '2022-08-11 08:20:57', 'type': '1', 'rate': '0.99988', 'amount': '157.59248663', 'total': '157.6114', 'orderNumber': None}
last record: {'globalTradeID': '60256425', 'tradeID': '60256425', 'date': '2022-08-11 08:23:20', 'type': '2', 'rate': '0.99952', 'amount': '57.24440949', 'total': '57.2719', 'orderNumber': None}
USDT_USDD: 100 records was written into db
new end: 2022-08-11 08:23:20
new end ts: 1660206199
done: False
Download data from 1660089600 to 1660206199
len response: 100
first record: {'globalTradeID': '60256329', 'tradeID': '60256329', 'date': '2022-08-11 08:20:57', 'type': '1', 'rate': '0.99988', 'amount': '157.59248663', 'total': '157.6114', 'orderNumber': None}
last record: {'globalTradeID': '60256425', 'tradeID': '60256425', 'date': '2022-08-11 08:23:20', 'type': '2', 'rate': '0.99952', 'amount': '57.24440949', 'total': '57.2719', 'orderNumber': None}
USDT_USDD: 100 records was 

len response: 100
first record: {'globalTradeID': '60256329', 'tradeID': '60256329', 'date': '2022-08-11 08:20:57', 'type': '1', 'rate': '0.99988', 'amount': '157.59248663', 'total': '157.6114', 'orderNumber': None}
last record: {'globalTradeID': '60256426', 'tradeID': '60256426', 'date': '2022-08-11 08:23:24', 'type': '2', 'rate': '0.9996', 'amount': '6.32946720', 'total': '6.332', 'orderNumber': None}
USDT_USDD: 100 records was written into db
new end: 2022-08-11 08:23:24
new end ts: 1660206203
done: False
Download data from 1660089600 to 1660206203
len response: 100
first record: {'globalTradeID': '60256329', 'tradeID': '60256329', 'date': '2022-08-11 08:20:57', 'type': '1', 'rate': '0.99988', 'amount': '157.59248663', 'total': '157.6114', 'orderNumber': None}
last record: {'globalTradeID': '60256426', 'tradeID': '60256426', 'date': '2022-08-11 08:23:24', 'type': '2', 'rate': '0.9996', 'amount': '6.32946720', 'total': '6.332', 'orderNumber': None}
USDT_USDD: 100 records was written 

KeyboardInterrupt: 

In [582]:
# Test PublicApi

api = PublicApiV2()
start_dt = "2022-08-15 00:00:00"
end_dt = "2022-08-15 18:00:00"

start_ts = api.date_to_unix_ts_in_utc(start_dt)
end_ts = api.date_to_unix_ts_in_utc(end_dt) 

symbol = "ETH_USDT"
interval = "MINUTE_1"

df = api.get_candles(symbol, interval, limit=500,  startTime=start_ts, endTime=end_ts)

display(df.shape)
display(df.head(2))

display(api.unix_ts_to_date(min(df["startTime"])))
display(api.unix_ts_to_date(max(df["startTime"])))



#print(api.get_price(symbol))
#print(api.get_orderbook(symbol, limit=500))
#print(api.get_trades(symbol, limit=500))


(500, 16)

,low,high,open,close,amount,quantity,buyTakerAmount,buyTakerQuantity,tradeCount,ts,weightedAverage,interval,startTime,closeTime,sellTakerAmount,sellTakerQuantity
0,1889.59,1890.56,1890.56,1889.59,753.95611,0.399,0.0,0.0,2,0,1889.614323,MINUTE_1,1660556460000,1660556519999,753.95611,0.399
1,1889.59,1889.59,1889.59,1889.59,0.00000,0.000,0.0,0.0,0,0,1889.614323,MINUTE_1,1660556520000,1660556579999,0.00000,0.000


'2022-08-15 09:41:00.000000'

'2022-08-15 18:00:00.000000'

In [630]:
class OrderbookProvider:
    DIVIDER = 1000
    
    def __init__(self):
        self.api = PublicApiV2()
    
    def get_scale(self, price):
        if  price > 1:
            price_len = len(str(int(price)))
            scale = int('1' + '0'* (price_len))/self.DIVIDER
        else:
            price_len = len(str(int(1/price)))
            scale = 1/(int('1' + '0'* (price_len))*self.DIVIDER)
        return scale
    
    def get_orderbook(self, symbol, auto_scale=False):
        price_data = self.api.get_price(symbol)
        price = float(price_data["price"])
        
        if auto_scale:
            scale = self.get_scale(price)
        else:
            scale=-1
        
        ob_data = self.api.get_orderbook(symbol, scale=scale, limit=150)
        
        asks_keys = ob_data["asks"][::2]
        asks_vals = map(float, ob_data["asks"][1::2])
        asks = dict(zip(asks_keys, asks_vals))


        bids_keys = ob_data["bids"][::2]
        bids_vals = map(float, ob_data["bids"][1::2])
        bids = dict(zip(bids_keys, bids_vals))
        
        orderbook = {
            "asks": asks,
            "bids": bids
        }
        return orderbook
    


In [631]:
# Test OrderbookProvider
api = PublicApiV2()


obp = OrderbookProvider()
symbol = "BTC_USDT"
ob = obp.get_orderbook(symbol)


print(api.get_price(symbol))
print("---------------")

print(ob["asks"])
print("---------------")
print(ob["bids"])

{'symbol': 'BTC_USDT', 'price': '23372.91', 'time': 1660852584233, 'dailyChange': '0.0041', 'ts': 1660852584238}
---------------
{'23373.05': 0.292609, '23374.21': 0.305, '23377.94': 0.305, '23379.1': 0.427473, '23379.42': 0.305, '23381.76': 0.347767, '23385.29': 0.305, '23387.94': 0.305, '23390.28': 0.305, '23393.89': 0.305, '23395.84': 0.243264, '23397.1': 0.153, '23397.27': 0.305, '23401.47': 0.305, '23403.7': 0.0533, '23403.82': 0.305, '23406.18': 0.305, '23408.53': 0.305, '23409.9': 0.1, '23410.88': 0.305, '23413.23': 0.305, '23417': 0.05293, '23418.91': 0.305, '23420.6': 0.314, '23421.83': 0.305, '23422.88': 0.482447, '23424.6': 0.305, '23425': 0.03, '23427.35': 0.305, '23430.79': 0.305, '23434.58': 0.305, '23437.44': 0.305, '23438.94': 0.000429, '23450.53': 0.473564, '23490': 2.67283, '23500': 0.001, '23509.54': 0.000505, '23517': 8e-05, '23517.34': 0.000404, '23534.19': 3.09, '23585': 0.002038, '23600': 0.058048, '23601': 0.01, '23611': 0.01, '23611.56': 0.007933, '23618': 0.01

In [629]:
1-23378.33/24643.62

0.0513435120327288

In [470]:
trades = api.get_trades(symbol, limit=1000)

In [471]:
import pandas as pd

In [486]:
df_t = pd.DataFrame(trades)
df_t["price"] = df_t["price"].astype(float)
df_t["quantity"] = df_t["quantity"].astype(float)
df_t["amount"] = df_t["amount"].astype(float)

In [487]:
df_t.head()

,id,price,quantity,amount,takerSide,ts,createTime
0,60142396,1874.97,0.011980,22.462141,SELL,1660828209468,1660828209463
1,60142395,1875.57,0.011782,22.097966,BUY,1660828182524,1660828182520
2,60142394,1874.54,0.013843,25.949257,BUY,1660828166312,1660828166307
3,60142393,1873.09,0.010869,20.358615,SELL,1660828140171,1660828140167
4,60142392,1874.20,0.010937,20.498125,BUY,1660828115220,1660828115215


In [130]:
host = "https://api.poloniex.com"

In [358]:
import time

1660767188000

In [ ]:
1660766853116
1660767188000
1660767082633730000
1660766948.308099

In [488]:
df_f = df_t[(df_t["ts"]<=endTime) & ( df_t["ts"] >= startTime)]

In [490]:
df_f["quantity"].sum()

3.129132

In [495]:
df_f.loc[df_f["takerSide"]=='BUY', "quantity"].sum()

0.38092499999999996

In [420]:
10/1890

0.005291005291005291

In [281]:
def get_scale(price):
    if  price > 1:
        price_len = len(str(int(price)))
        scale = int(int('1' + '0'* (price_len))/1000)
    else:
        price_len = len(str(int(1/price)))
        scale = 1/(int('1' + '0'* (price_len))*1000)
    return scale

In [350]:
symbol = "BTC_USDT"
path = "/markets/{0}/price".format(symbol)
response = requests.get(host + path).json()


price = float(response["price"])
price

23275.58

In [351]:
response

{'symbol': 'BTC_USDT',
 'price': '23275.58',
 'time': 1660764303941,
 'dailyChange': '-0.0288',
 'ts': 1660764303947}

In [345]:
scale = get_scale(price)
scale

100.0

In [346]:
path = "/markets/{0}/orderBook?limit=150&scale={1}".format(symbol, scale)
response = requests.get(host + path).json()
if "code" in response:
    print(response)
else:

    asks_keys = response["asks"][::2]
    asks_vals = map(float, response["asks"][1::2])
    asks = dict(zip(asks_keys, asks_vals))


    bids_keys = response["bids"][::2]
    bids_vals = map(float, response["bids"][1::2])
    bids = dict(zip(bids_keys, bids_vals))

In [347]:
print(asks)
print("----------")
print(bids)

{'23700': 8.110453, '23800': 3.984519, '23900': 1.57468, '24000': 0.028727, '24100': 0.108521, '24200': 0.039439, '24300': 0.04295, '24400': 0.154606, '24500': 0.751562, '24600': 0.028378, '24700': 0.527226, '24800': 0.067483, '24900': 0.836699, '25000': 2.863543, '25100': 0.928284, '25200': 0.727149, '25300': 0.886503, '25400': 0.207176, '25500': 1.237196, '25600': 0.038974, '25700': 0.01066, '25800': 0.555619, '25900': 0.130361, '26000': 0.904372, '26100': 0.100197, '26200': 0.028389, '26300': 2.023602, '26400': 0.019397, '26500': 0.004291, '26600': 0.088779, '26700': 0.002036, '26800': 0.03327, '26900': 0.013921, '27000': 0.526575, '27100': 0.113385, '27200': 0.317611, '27300': 0.463942, '27400': 0.016935, '27500': 0.494536, '27600': 0.872462, '27700': 0.653384, '27800': 0.078911, '27900': 0.502072, '28000': 0.040814, '28100': 0.203056, '28200': 0.795779, '28300': 0.014548, '28400': 0.000352, '28500': 0.198322, '28600': 0.018484, '28800': 0.000694, '28900': 0.120959, '29000': 0.0498

In [338]:
asks

{'0.0000002199': 1000.0,
 '0.0000002200': 1000.0,
 '0.0000002300': 2542.0,
 '0.0000002500': 1000.0,
 '0.0000003300': 2840.0,
 '0.0000003400': 5476.0,
 '0.0000003700': 1005.0,
 '0.0000005200': 3376.0,
 '0.0000006300': 3673.0,
 '0.0000007300': 2074.0,
 '0.0000099010': 11055.0,
 '0.0000100000': 1000.0,
 '0.0000149994': 8000.0,
 '0.0000149995': 10000.0,
 '0.0000149996': 10000.0,
 '0.0000149997': 10000.0,
 '0.0000149998': 10000.0,
 '0.0000149999': 10000.0,
 '0.0000150000': 10294.0,
 '0.0000160000': 36044.0,
 '0.0000170000': 40800.0,
 '0.0000199993': 1000.0,
 '0.0000199994': 10000.0,
 '0.0000199995': 10000.0,
 '0.0000199996': 10000.0,
 '0.0000199997': 10000.0,
 '0.0000199998': 10000.0,
 '0.0000199999': 10000.0,
 '0.0000200000': 12384.0,
 '0.0000250000': 1505.0,
 '0.0002000000': 5000.0,
 '0.0020000000': 1000.0,
 '0.0028800000': 1000.0}

In [339]:
bids

{'0.0000001520': 5568.0,
 '0.0000001450': 7982.0,
 '0.0000001300': 1000.0,
 '0.0000001201': 4941.0,
 '0.0000001200': 1000.0,
 '0.0000001100': 1000.0,
 '0.0000001050': 1100.0,
 '0.0000001008': 14880.0,
 '0.0000000808': 44644.0,
 '0.0000000608': 39222.0,
 '0.0000000600': 2563.0,
 '0.0000000500': 5145.0,
 '0.0000000400': 1610.0,
 '0.0000000300': 1845.0,
 '0.0000000100': 1985.0,
 '0.0000000034': 8469.0,
 '0.0000000010': 990099.0,
 '0.0000000003': 154963.0,
 '0.0000000001': 7142857.0,
 '0.0000000000': 193333333.0}

In [315]:
def get_scale(price):
    if  price > 1:
        price_len = len(str(int(price)))
        scale = int('1' + '0'* (price_len))/1000
        
    else:
        price_len = len(str(int(1/price)))
        scale = 1/(int('1' + '0'* (price_len))*1000)
    return scale

In [320]:
v = 99
scale = get_scale(v)

round(v/scale)*scale

99.0

In [321]:
scale

0.1

In [323]:
scale/v

0.00101010101010101